# Load the data

plan:
    1. aggregate by sum() everything possible
    2. get proportions of health and indicators of dead players
    3. create features for coordinates
    4. generate something for level
    5. dummy variables for game_mode
    6. ....scaling will be done in the training part (CV)

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']= 100
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [176]:
# load the data and combine with test data to perform some of the trasformations (without standartization) on all data at the same time
train_df = pd.read_csv('train_features_plus.csv', index_col='match_id_hash')
test_df = pd.read_csv('test_features_plus.csv', index_col='match_id_hash')
idx_split = train_df.shape[0]
full_df = pd.concat([train_df, test_df])
new_features = pd.DataFrame(index=full_df.index)
full_df.head(2)

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed,d5_ability_level,d5_max_hero_hit,d5_purchase_count,d5_count_ability_use,d5_damage_dealt,d5_damage_received
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,...,0,0,0,0,0,164,6,4,2332,681
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,0,0,0,0,4,164,11,7,2308,2154


# Building feautures on hero_ids

In [155]:
target = pd.read_csv(r'C:\Users\Tatiana\Documents\Programming\MLcourse competition\target_plus.csv', index_col = 'match_id_hash')['radiant_win']

full_train = train_df.merge(target, how='outer', left_index=True, right_index=True)
full_train.shape

(39675, 306)

In [101]:
hero_id_list = ['r{}_hero_id'.format(i) for i in range(1,6)] + ['d{}_hero_id'.format(i) for i in range(1,6)]
hero_ids = full_train[hero_id_list+['radiant_win']]
hero_ids_rad_win = hero_ids[hero_ids['radiant_win'] == True]
hero_ids_rad_lose = hero_ids[hero_ids['radiant_win'] == False]
# winning
winning_hero_ids1 = hero_ids_rad_win[['r{}_hero_id'.format(i) for i in range(1,6)]]
winning_hero_ids2 = hero_ids_rad_lose[['d{}_hero_id'.format(i) for i in range(1,6)]]
# losing
losing_hero_ids1 = hero_ids_rad_win[['d{}_hero_id'.format(i) for i in range(1,6)]]
losing_hero_ids2 = hero_ids_rad_lose[['r{}_hero_id'.format(i) for i in range(1,6)]]

In [102]:
# renaming columns
winning_hero_ids1.rename(columns = {'r1_hero_id':'1_id', 'r2_hero_id':'2_id', 
                              'r3_hero_id':'3_id','r4_hero_id':'4_id',
                                    'r5_hero_id':'5_id'}, inplace = True)
winning_hero_ids2.rename(columns = {'d1_hero_id':'1_id', 'd2_hero_id':'2_id', 
                              'd3_hero_id':'3_id','d4_hero_id':'4_id',
                                    'd5_hero_id':'5_id'}, inplace = True)

losing_hero_ids1.rename(columns = {'d1_hero_id':'1_id', 'd2_hero_id':'2_id', 
                              'd3_hero_id':'3_id','d4_hero_id':'4_id',
                                    'd5_hero_id':'5_id'}, inplace = True)
losing_hero_ids2.rename(columns = {'r1_hero_id':'1_id', 'r2_hero_id':'2_id', 
                              'r3_hero_id':'3_id','r4_hero_id':'4_id',
                                    'r5_hero_id':'5_id'}, inplace = True)

In [103]:
winning_hero_ids = pd.concat([winning_hero_ids1, winning_hero_ids2], axis=0)
winning_hero_ids.shape

(39675, 5)

In [104]:
losing_hero_ids = pd.concat([losing_hero_ids1, losing_hero_ids2], axis=0)
losing_hero_ids.shape

(39675, 5)

In [105]:
winning_hero_counts = winning_hero_ids['1_id'].value_counts().sort_index() + winning_hero_ids['2_id'].value_counts().sort_index() + winning_hero_ids['3_id'].value_counts().sort_index() + winning_hero_ids['4_id'].value_counts().sort_index() + winning_hero_ids['5_id'].value_counts().sort_index()
winning_hero_counts = winning_hero_counts.sort_values()
winning_hero_dict = winning_hero_counts.to_dict()

In [106]:
losing_hero_counts = losing_hero_ids['1_id'].value_counts().sort_index() + losing_hero_ids['2_id'].value_counts().sort_index() + losing_hero_ids['3_id'].value_counts().sort_index() + losing_hero_ids['4_id'].value_counts().sort_index() + losing_hero_ids['5_id'].value_counts().sort_index()
losing_hero_counts = losing_hero_counts.sort_values()
losing_hero_dict = losing_hero_counts.to_dict()

In [107]:
hero_counts_win_minus_lose = winning_hero_counts.sort_index() - losing_hero_counts.sort_index()
diff_hero_dict = hero_counts_win_minus_lose.to_dict()


In [177]:
for col in hero_id_list:
    full_df[col] = full_df[col].map(diff_hero_dict)
    train_df[col] = train_df[col].map(diff_hero_dict)
    test_df[col] = test_df[col].map(diff_hero_dict)

# Building tree based on location (+ (optionally) experience, health, gold)

In [ ]:
## tree with extended features

# health = ['r{}_health'.format(x) for x in range(1,6)] +['d{}_health'.format(x) for x in range(1,6)]
# max_health = ['r{}_health'.format(x) for x in range(1,6)] +['d{}_health'.format(x) for x in range(1,6)]
# x = ['r{}_x'.format(x) for x in range(1,6)] +['d{}_x'.format(x) for x in range(1,6)]
# y = ['r{}_y'.format(x) for x in range(1,6)] +['d{}_y'.format(x) for x in range(1,6)]
# # xp = ['r{}_xp'.format(x) for x in range(1,6)] +['d{}_xp'.format(x) for x in range(1,6)]
# # gold = ['r{}_gold'.format(x) for x in range(1,6)] +['d{}_gold'.format(x) for x in range(1,6)]

# train_positions = train_df[health+max_health+x+y+xp]
# train_positions.head(2)

In [ ]:

x = ['r{}_x'.format(x) for x in range(1,6)] +['d{}_x'.format(x) for x in range(1,6)]
y = ['r{}_y'.format(x) for x in range(1,6)] +['d{}_y'.format(x) for x in range(1,6)]
hero_id = ['r{}_hero_id'.format(x) for x in range(1,6)] +['d{}_hero_id'.format(x) for x in range(1,6)]

train_positions = train_df[x+y]
train_positions.head(2)

In [ ]:
target = pd.read_csv(r'C:\Users\Tatiana\Documents\Programming\MLcourse competition\train_targets.csv', index_col = 'match_id_hash')
y_t = target['radiant_win']
y_t = y_t.apply(lambda x: int(x)).values



In [ ]:
from sklearn.model_selection import train_test_split

X = train_positions.values

X_train, X_test, y_train, y_test = train_test_split(X, y_t, test_size=0.30, random_state=150)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

cls = RandomForestClassifier(max_features = 'auto', max_depth = 16, n_estimators = 500, min_samples_leaf=2)
cls.fit(X_train,y_train)


In [ ]:
cls.score(X_test,y_test)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
predicted = cls.predict_proba(X_test)[:,1]
roc_auc_score(y_test,predicted)

In [ ]:
# positions_full = full_df[health+max_health+x+y+xp]
# positions = cls.predict(positions_full)
# new_features['positions'] = positions

In [ ]:

positions_full = full_df[x+y]
positions = cls.predict(positions_full)
# new_features['positions'] = positions

# Transform health

In [178]:
def are_dead(x):
    ans = 0 
    for el in x:
        if el == 0:
            ans += 1
    return ans

In [179]:
# add number of dead players at the end of the game in each team

d_health = ['d{}_health'.format(i) for i in range(1,6)]
new_features['d_dead'] = full_df[d_health].apply(are_dead, axis = 1)
r_health = ['r{}_health'.format(i) for i in range(1,6)]
new_features['r_dead'] = full_df[r_health].apply(are_dead, axis = 1)

In [180]:
# calculate the proportion of health
percentage_health = 0
for j in range(5):
    max_health = '{}_max_health'.format(d_health[j].split('_')[0])
    health = 'd{}_health'.format(j+1)
    ph = full_df[health]/full_df[max_health]
    percentage_health += 1/5*ph

new_features['d_health_avg'] = percentage_health

percentage_health = 0
for j in range(5):
    max_health = '{}_max_health'.format(r_health[j].split('_')[0])
    health = 'r{}_health'.format(j+1)
    ph = full_df[health]/full_df[max_health]
    percentage_health += 1/5*ph

new_features['r_health_avg'] = percentage_health

In [181]:
new_features.head()

,d_dead,r_dead,d_health_avg,r_health_avg
match_id_hash,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.954967,0.715500
b9c57c450ce74a2af79c9ce96fac144d,0,0,0.764778,0.846735
6db558535151ea18ca70a6892197db41,0,0,0.989032,0.989333
46a0ddce8f7ed2a8d9bd5edcbb925682,0,0,0.869557,0.955818
b1b35ff97723d9b7ade1c9c3cf48f770,0,0,0.808481,0.735127


# Transform coordinates

In [129]:
def in_opponent_base(x,y,opponent = 'Radiant'):

    '''
returns indicator function of whether the player is in the base of the opponent
opponent = {'Radiant', 'Dire'} - opponent team
    
    '''
#     x = coordinates[0]
#     y = coordinates[1]
    radiant_base_x = 96
    radiant_base_y = 100

    dire_base_x = 156
    dire_base_y = 156

    if opponent == 'Radiant':
        if x <= radiant_base_x and y <= radiant_base_y:
            return 1
        else:
            return 0
    elif opponent == 'Dire':
        if x >= dire_base_x and y >= dire_base_y:
            return 1
        else:
            return 0
    else:
        return NaN

In [161]:
r_x = ['r{}_x'.format(j) for j in range(1,6)]
r_y = ['r{}_y'.format(j) for j in range(1,6)]
d_x = ['d{}_x'.format(j) for j in range(1,6)]
d_y = ['d{}_y'.format(j) for j in range(1,6)]

In [ ]:
# # getting indicator function for each player

# for j in range(5):
#     rx = r_x[j]
#     ry = r_y[j]
#     dx = d_x[j]
#     dy = d_y[j]
#     new_features['r{}_in_d_base'.format(j+1)] = full_df.loc[:,[rx,ry]].apply(lambda x: in_opponent_base(x = x[rx], y = x[ry], opponent = 'Dire'), axis = 1)
#     new_features['d{}_in_r_base'.format(j+1)] = full_df.loc[:,[dx,dy]].apply(lambda x: in_opponent_base(x = x[dx], y = x[dy], opponent = 'Radiant'), axis = 1)
#     new_features['r{}_in_r_base'.format(j+1)] = full_df.loc[:,[rx,ry]].apply(lambda x: in_opponent_base(x = x[rx], y = x[ry], opponent = 'Radiant'), axis = 1)
#     new_features['d{}_in_d_base'.format(j+1)] = full_df.loc[:,[dx,dy]].apply(lambda x: in_opponent_base(x = x[dx], y = x[dy], opponent = 'Dire'), axis = 1)
    
#     new_features['r{}_in_d_base_hero_id'.format(j+1)] = new_features['r{}_in_d_base'.format(j+1)]*new_features['r{}_hero_id'.format(j+1)]
#     new_features['d{}_in_r_base_hero_id'.format(j+1)] = new_features['d{}_in_r_base'.format(j+1)]*new_features['d{}_hero_id'.format(j+1)]
    
#     new_features['r{}_in_d_base_count_ability_use'.format(j+1)] = new_features['r{}_in_d_base'.format(j+1)]*new_features['r{}_count_ability_use'.format(j+1)]
#     new_features['d{}_in_r_base_count_ability_use'.format(j+1)] = new_features['d{}_in_r_base'.format(j+1)]*new_features['d{}_count_ability_use'.format(j+1)]
    
#     new_features['r{}_in_d_max_hero_hit'.format(j+1)] = new_features['r{}_in_d_base'.format(j+1)]*new_features['r{}_max_hero_hit'.format(j+1)]
#     new_features['d{}_in_r_max_hero_hit'.format(j+1)] = new_features['d{}_in_r_base'.format(j+1)]*new_features['d{}_max_hero_hit'.format(j+1)]
    

In [182]:
# getting indicator function for each player
# !!!!!!!!!!!!!!!!!!!!!!! Do not run if using the small set only

for j in range(5):
    rx = r_x[j]
    ry = r_y[j]
    dx = d_x[j]
    dy = d_y[j]
    full_df['r{}_in_d_base'.format(j+1)] = full_df.loc[:,[rx,ry]].apply(lambda x: in_opponent_base(x = x[rx], y = x[ry], opponent = 'Dire'), axis = 1)
    full_df['d{}_in_r_base'.format(j+1)] = full_df.loc[:,[dx,dy]].apply(lambda x: in_opponent_base(x = x[dx], y = x[dy], opponent = 'Radiant'), axis = 1)
    full_df['r{}_in_r_base'.format(j+1)] = full_df.loc[:,[rx,ry]].apply(lambda x: in_opponent_base(x = x[rx], y = x[ry], opponent = 'Radiant'), axis = 1)
    full_df['d{}_in_d_base'.format(j+1)] = full_df.loc[:,[dx,dy]].apply(lambda x: in_opponent_base(x = x[dx], y = x[dy], opponent = 'Dire'), axis = 1)
    
    full_df['r{}_in_d_base_hero_id'.format(j+1)] = full_df['r{}_in_d_base'.format(j+1)]*full_df['r{}_hero_id'.format(j+1)]
    full_df['d{}_in_r_base_hero_id'.format(j+1)] = full_df['d{}_in_r_base'.format(j+1)]*full_df['d{}_hero_id'.format(j+1)]
    
    full_df['r{}_in_d_base_count_ability_use'.format(j+1)] = full_df['r{}_in_d_base'.format(j+1)]*full_df['r{}_count_ability_use'.format(j+1)]
    full_df['d{}_in_r_base_count_ability_use'.format(j+1)] = full_df['d{}_in_r_base'.format(j+1)]*full_df['d{}_count_ability_use'.format(j+1)]
    
    full_df['r{}_in_d_max_hero_hit'.format(j+1)] = full_df['r{}_in_d_base'.format(j+1)]*full_df['r{}_max_hero_hit'.format(j+1)]
    full_df['d{}_in_r_max_hero_hit'.format(j+1)] = full_df['d{}_in_r_base'.format(j+1)]*full_df['d{}_max_hero_hit'.format(j+1)]
    
    full_df['r{}_in_d_damage_dealt'.format(j+1)] = full_df['r{}_in_d_base'.format(j+1)]*full_df['r{}_damage_dealt'.format(j+1)]
    full_df['d{}_in_r_damage_dealt'.format(j+1)] = full_df['d{}_in_r_base'.format(j+1)]*full_df['d{}_damage_dealt'.format(j+1)]
    
    full_df['r{}_in_r_damage_received'.format(j+1)] = full_df['r{}_in_r_base'.format(j+1)]*full_df['r{}_damage_received'.format(j+1)]
    full_df['d{}_in_d_damage_received'.format(j+1)] = full_df['d{}_in_d_base'.format(j+1)]*full_df['d{}_damage_received'.format(j+1)]

In [183]:
# getting total amount of players of each team at each base

r_in_d_base = pd.Series(0,index = full_df.index)
d_in_r_base = pd.Series(0,index = full_df.index)
r_in_r_base = pd.Series(0,index = full_df.index)
d_in_d_base = pd.Series(0,index = full_df.index)

for j in range(5):
    rx = r_x[j]
    ry = r_y[j]
    dx = d_x[j]
    dy = d_y[j]
    r_in_d_base += full_df.loc[:,[rx,ry]].apply(lambda x: in_opponent_base(x = x[rx], y = x[ry], opponent = 'Dire'), axis = 1)
    d_in_r_base += full_df.loc[:,[dx,dy]].apply(lambda x: in_opponent_base(x = x[dx], y = x[dy], opponent = 'Radiant'), axis = 1)
    r_in_r_base += full_df.loc[:,[rx,ry]].apply(lambda x: in_opponent_base(x = x[rx], y = x[ry], opponent = 'Radiant'), axis = 1)
    d_in_d_base += full_df.loc[:,[dx,dy]].apply(lambda x: in_opponent_base(x = x[dx], y = x[dy], opponent = 'Dire'), axis = 1)

In [184]:
new_features['r_in_d_base'] = r_in_d_base
new_features['d_in_r_base'] = d_in_r_base
new_features['r_in_r_base'] = r_in_d_base
new_features['d_in_d_base'] = d_in_r_base
new_features.head()

,d_dead,r_dead,d_health_avg,r_health_avg,r_in_d_base,d_in_r_base,r_in_r_base,d_in_d_base
match_id_hash,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.954967,0.715500,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,0,0,0.764778,0.846735,0,0,0,0
6db558535151ea18ca70a6892197db41,0,0,0.989032,0.989333,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,0,0,0.869557,0.955818,0,0,0,0
b1b35ff97723d9b7ade1c9c3cf48f770,0,0,0.808481,0.735127,0,0,0,0


In [185]:
new_features.columns

Index(['d_dead', 'r_dead', 'd_health_avg', 'r_health_avg', 'r_in_d_base',
       'd_in_r_base', 'r_in_r_base', 'd_in_d_base'],
      dtype='object')

# Aggregate features within the team

In [186]:
# Create a feature that is sum of r1 + ... + r5 for all r features, and same for all d features.
# Features that do not make sence are removed.
new_feats = [i.replace("1","")  for i in full_df.columns.values if (i.startswith('r1')==1 or i.startswith('d1')==1)]
rem_feat = ['r_health', 'r_max_health','r_level', 'r_x', 'r_y', 'r_teamfight_participation', 'd_health','d_max_health', 'd_level', 'd_x', 'd_y', 'd_teamfight_participation']
for feat in rem_feat:
    new_feats.remove(feat)

for feature in new_feats:
    feat_names = [(feature[0] + '{}_'.format(i) +feature[2:]) for i in range(1,6)]
    new_features[feature] = full_df.loc[:,feat_names].sum(axis=1)
print(new_features.columns)
new_features.head()

Index(['d_dead', 'r_dead', 'd_health_avg', 'r_health_avg', 'r_in_d_base',
       'd_in_r_base', 'r_in_r_base', 'd_in_d_base', 'r_hero_id', 'r_kills',
       'r_deaths', 'r_assists', 'r_denies', 'r_gold', 'r_lh', 'r_xp',
       'r_max_mana', 'r_stuns', 'r_creeps_stacked', 'r_camps_stacked',
       'r_rune_pickups', 'r_firstblood_claimed', 'r_towers_killed',
       'r_roshans_killed', 'r_obs_placed', 'r_sen_placed', 'r_ability_level',
       'r_max_hero_hit', 'r_purchase_count', 'r_count_ability_use',
       'r_damage_dealt', 'r_damage_received', 'd_hero_id', 'd_kills',
       'd_deaths', 'd_assists', 'd_denies', 'd_gold', 'd_lh', 'd_xp',
       'd_max_mana', 'd_stuns', 'd_creeps_stacked', 'd_camps_stacked',
       'd_rune_pickups', 'd_firstblood_claimed', 'd_towers_killed',
       'd_roshans_killed', 'd_obs_placed', 'd_sen_placed', 'd_ability_level',
       'd_max_hero_hit', 'd_purchase_count', 'd_count_ability_use',
       'd_damage_dealt', 'd_damage_received', 'r_in_d_base_hero_id',
 

,d_dead,r_dead,d_health_avg,r_health_avg,r_in_d_base,d_in_r_base,r_in_r_base,d_in_d_base,r_hero_id,r_kills,...,r_in_d_base_hero_id,d_in_r_base_hero_id,r_in_d_base_count_ability_use,d_in_r_base_count_ability_use,r_in_d_max_hero_hit,d_in_r_max_hero_hit,r_in_d_damage_dealt,d_in_r_damage_dealt,r_in_r_damage_received,d_in_d_damage_received
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.954967,0.715500,0,0,0,0,-355,0,...,0,0,0,0,0,0,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,0,0,0.764778,0.846735,0,0,1,1,341,16,...,0,0,0,0,0,0,0,0,4857,2154
6db558535151ea18ca70a6892197db41,0,0,0.989032,0.989333,0,0,0,2,3,0,...,0,0,0,0,0,0,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,0,0,0.869557,0.955818,0,0,0,0,559,6,...,0,0,0,0,0,0,0,0,0,0
b1b35ff97723d9b7ade1c9c3cf48f770,0,0,0.808481,0.735127,0,0,0,0,735,2,...,0,0,0,0,0,0,0,0,0,0


# Aggreagate the level of players

In [187]:
d_levels = ['d{}_level'.format(j) for j in range(1,6)] 
d_avg_level = full_df.loc[:,d_levels].mean(axis = 1)
d_min_level = full_df.loc[:,d_levels].min(axis = 1)
d_max_level = full_df.loc[:,d_levels].max(axis = 1)
new_features['d_avg_level'] = d_avg_level
new_features['d_min_level'] = d_min_level
new_features['d_max_level'] = d_max_level


r_levels = ['r{}_level'.format(j) for j in range(1,6)] 
r_avg_level = full_df.loc[:,r_levels].mean(axis = 1)
r_min_level = full_df.loc[:,r_levels].min(axis = 1)
r_max_level = full_df.loc[:,r_levels].max(axis = 1)
new_features['r_avg_level'] = r_avg_level
new_features['r_min_level'] = r_min_level
new_features['r_max_level'] = r_max_level

new_features.head(2)

,d_dead,r_dead,d_health_avg,r_health_avg,r_in_d_base,d_in_r_base,r_in_r_base,d_in_d_base,r_hero_id,r_kills,...,r_in_d_damage_dealt,d_in_r_damage_dealt,r_in_r_damage_received,d_in_d_damage_received,d_avg_level,d_min_level,d_max_level,r_avg_level,r_min_level,r_max_level
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.954967,0.715500,0,0,0,0,-355,0,...,0,0,0,0,2.6,2,3,1.8,1,2
b9c57c450ce74a2af79c9ce96fac144d,0,0,0.764778,0.846735,0,0,1,1,341,16,...,0,0,4857,2154,6.0,4,7,7.4,6,9


# Team fights participation

In [188]:
# full_df[(full_df.r1_health < full_df.r1_max_health*0.5) & (full_df.r1_health > 0)].r1_teamfight_participation.hist(bins =100)
# full_df[full_df.r1_health == full_df.r1_max_health].r1_teamfight_participation.hist(bins =100)
# full_df[full_df.r1_health == 0].r1_teamfight_participation.hist(bins =100)
r_teamfight = ['r{}_teamfight_participation'.format(j) for j in range(1,6)] 
for col in r_teamfight:
    full_df[full_df[col] >1][col] = 1

d_teamfight = ['d{}_teamfight_participation'.format(j) for j in range(1,6)] 
for col in d_teamfight:
    full_df[full_df[col] >1][col] = 1

# dummy variables encoding

In [ ]:
# game_mode = pd.get_dummies(full_df['game_mode'])
# game_mode.columns

In [ ]:
# game_mode_col = []
# for name in game_mode.columns:
#     name = 'game_mode_{}'.format(name)
#     game_mode_col.append(name) 
# game_mode.columns = game_mode_col
# game_mode.head(2)

In [ ]:
# new_features = new_features.merge(game_mode, how='inner', left_index=True, right_index=True)
# new_features.head(2)

# Adding old features and recording to files

In [189]:
# add other useful features (not team-specific) to the new variables
# !!!!!!!!!!!!!!!!! Do not sun if using the small date set
col_to_drop = []
for i in range(1,6):
#     col_to_drop.append('r{}_hero_id'.format(i))
    col_to_drop.append('r{}_x'.format(i))
    col_to_drop.append('r{}_y'.format(i))
#     col_to_drop.append('d{}_hero_id'.format(i))
    col_to_drop.append('d{}_x'.format(i))
    col_to_drop.append('d{}_y'.format(i))

new_features = new_features.merge(full_df.drop(columns = col_to_drop), how='inner', left_index=True, right_index=True)
new_features.head(2)

,d_dead,r_dead,d_health_avg,r_health_avg,r_in_d_base,d_in_r_base,r_in_r_base,d_in_d_base,r_hero_id,r_kills,...,r5_in_d_base_hero_id,d5_in_r_base_hero_id,r5_in_d_base_count_ability_use,d5_in_r_base_count_ability_use,r5_in_d_max_hero_hit,d5_in_r_max_hero_hit,r5_in_d_damage_dealt,d5_in_r_damage_dealt,r5_in_r_damage_received,d5_in_d_damage_received
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.954967,0.715500,0,0,0,0,-355,0,...,0,0,0,0,0,0,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,0,0,0.764778,0.846735,0,0,1,1,341,16,...,0,0,0,0,0,0,0,0,0,2154


In [190]:
new_features['r_gold_sq'] = new_features['r_gold'].apply(lambda x: x**2)
new_features['d_gold_sq'] = new_features['d_gold'].apply(lambda x: x**2)
new_features['r_xp_sq'] = new_features['r_xp'].apply(lambda x: x**2)
new_features['d_xp_sq'] = new_features['d_xp'].apply(lambda x: x**2)
new_features['r_xp_base_d'] = new_features['r_xp']*new_features['r_in_d_base']
new_features['d_xp_base_r'] = new_features['d_xp']*new_features['d_in_r_base']

In [172]:
# creating smaller datset
new_features = new_features.merge(full_df[['chat_len','game_time','game_mode','lobby_type']], how='inner', left_index=True, right_index=True)
new_features.head(2)

,d_dead,r_dead,d_health_avg,r_health_avg,r_in_d_base,d_in_r_base,r_in_r_base,d_in_d_base,r_hero_id,r_kills,...,r_gold_sq,d_gold_sq,r_xp_sq,d_xp_sq,r_xp_base_d,d_xp_base_r,chat_len_y,game_time_y,game_mode_y,lobby_type_y
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.954967,0.715500,0,0,0,0,-355,0,...,4149369,15586704,3964081,9511056,0,0,11,155,22,7
b9c57c450ce74a2af79c9ce96fac144d,0,0,0.764778,0.846735,0,0,1,1,341,16,...,330221584,172764736,308564356,167055625,0,0,10,658,4,0


In [191]:
log_tags_r = ['r_kills', 'r_deaths', 'r_assists','r_denies', 'r_gold', 'r_lh', 'r_xp', 'r_max_mana', 'r_creeps_stacked',
               'r_camps_stacked','r_rune_pickups', 'r_sen_placed', 'r_obs_placed', 'r_max_hero_hit', 'r_purchase_count',
           'r_count_ability_use', 'r_damage_dealt', 'r_damage_received']
new_features[log_tags_r].describe()


,r_kills,r_deaths,r_assists,r_denies,r_gold,r_lh,r_xp,r_max_mana,r_creeps_stacked,r_camps_stacked,r_rune_pickups,r_sen_placed,r_obs_placed,r_max_hero_hit,r_purchase_count,r_count_ability_use,r_damage_dealt,r_damage_received
count,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,49675.000000,4.967500e+04,49675.000000
mean,15.553880,16.314625,23.558067,31.346412,35762.520523,319.795350,41362.091213,3619.058157,4.040544,1.331414,21.517685,3.981842,6.339547,1158.575279,122.443704,331.533347,2.135473e+05,49995.669633
std,13.080692,13.854083,23.124927,19.541436,29545.874844,275.691661,35107.147445,1551.673669,7.118989,1.943970,15.137301,5.861790,5.060419,675.467353,65.310583,429.331710,2.223128e+05,44961.071090
min,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,1250.690000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000e+00,0.000000
25%,4.000000,5.000000,5.000000,17.000000,11792.000000,102.000000,12528.000000,2318.690000,0.000000,0.000000,10.000000,0.000000,2.000000,640.000000,69.000000,89.000000,4.544600e+04,15257.500000
50%,13.000000,13.000000,17.000000,29.000000,28817.000000,250.000000,31244.000000,3356.690000,1.000000,1.000000,19.000000,2.000000,5.000000,1087.000000,116.000000,226.000000,1.440440e+05,38237.000000
75%,24.000000,25.000000,36.500000,44.000000,52958.000000,472.000000,62650.500000,4623.191500,5.000000,2.000000,30.000000,5.000000,10.000000,1552.000000,168.000000,421.000000,3.144945e+05,72372.500000
max,84.000000,86.000000,164.000000,138.000000,211676.000000,2535.000000,146996.000000,11124.698000,139.000000,21.000000,148.000000,88.000000,31.000000,7734.000000,410.000000,7879.000000,3.101739e+06,446546.000000


In [192]:
import math

for col in col_for_log:
    new_log_index = 'log_{}'.format(col)
#     print(new_features[col].min())
    new_log_col = new_features[col].apply(lambda x: math.log(x+1))
    new_features[new_log_index] = new_log_col
    
new_features.head()

,d_dead,r_dead,d_health_avg,r_health_avg,r_in_d_base,d_in_r_base,r_in_r_base,d_in_d_base,r_hero_id,r_kills,...,log_r_purchase_count,log_d_purchase_count,log_r_count_ability_use,log_d_count_ability_use,log_r_damage_dealt,log_d_damage_dealt,log_r_damage_received,log_d_damage_received,log_chat_len,log_game_time
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.954967,0.715500,0,0,0,0,-355,0,...,3.526361,3.583519,2.079442,2.484907,9.075322,9.153347,8.353026,7.349874,2.484907,5.049856
b9c57c450ce74a2af79c9ce96fac144d,0,0,0.764778,0.846735,0,0,1,1,341,16,...,4.653960,4.488636,4.553877,4.634729,11.152902,10.754044,9.961945,10.104794,2.397895,6.490724
6db558535151ea18ca70a6892197db41,0,0,0.989032,0.989333,0,0,0,2,3,0,...,3.091042,2.944439,0.693147,0.693147,4.262680,4.234107,4.234107,4.262680,0.000000,3.091042
46a0ddce8f7ed2a8d9bd5edcbb925682,0,0,0.869557,0.955818,0,0,0,0,559,6,...,4.276666,4.304065,4.753590,4.418841,10.827389,11.142325,9.632138,10.034910,1.609438,6.357842
b1b35ff97723d9b7ade1c9c3cf48f770,0,0,0.808481,0.735127,0,0,0,0,735,2,...,3.784190,3.713572,4.477337,4.262680,10.543076,10.900879,9.672060,9.148997,1.386294,6.118097


In [193]:
train = new_features.iloc[:idx_split, :]
test = new_features.iloc[idx_split:, :]

In [194]:
train.to_csv('new_feat_train.csv')
test.to_csv('new_feat_test.csv')

In [195]:
test.shape

(10000, 471)

In [152]:
train.to_csv('new_feat_train_small.csv')
test.to_csv('new_feat_test_small.csv')

In [153]:
test.shape

(10000, 120)